## Topic Modeling with Genism

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

#메일의 순수한 내용으로만 재구성
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

In [2]:
# 필요한 library들을 import
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
import re

In [3]:
def tokenize(text):
    min_length = 3 #최소 단어 크기를 3으로 설정
    words = map(lambda word: word.lower(), word_tokenize(text))    # nltk의 tokenizer를 이용해서 word 추출 -> 소문자로 변환
    words = [word for word in words if word not in cachedStopWords] # stopwords 제외
    tokens = (list(map(lambda token: PorterStemmer().stem(token),words)))
    p = re.compile('[a-zA-Z]+');  #'[a-zA-Z]+' => 알파벳으로 이루어진 단어들만 추출
    filtered_tokens = list(filter (lambda token: p.match(token) and len(token) >= min_length,tokens))
    return filtered_tokens

In [4]:
docs = [tokenize(doc) for doc in newsgroups_train.data]

In [5]:
print(newsgroups_train.data[0])
print(docs[0])

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
['notic', 'save', 'model', 'map', 'plane', 'posit', 'care', 'file', 'reload', 'restart', 'given', 'default', 'posit', 'orient', 'save', 'file', 'positions/orient', 'preserv', 'anyon', 'know', 'inform', 'store', 'file', 'noth', 'explicitli', 'said', 'manual', 'save', 'textur', 'rule', 'file', 'like', 'abl', 'read', 'textur', 'rule', 'inform', 'anyon', 'format', 'file', 'file', 'format', 'a

In [6]:
from gensim.corpora import Dictionary

dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

dictionary.filter_extremes(no_below=3, no_above=0.5)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 19839
Number of unique words after removing rare and common words: 5630


In [7]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 5630
Number of documents: 2034


In [8]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 25
chunksize = 500 
passes = 20 
iterations = 40
eval_every = 1 

temp = dictionary[0]  
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

Wall time: 42.2 s


In [9]:
# pip install pyldavis
import pyLDAvis
import pyLDAvis.gensim as gensimvis
pyLDAvis.enable_notebook()


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [11]:
pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.206581  0.053404       1        1  28.809437
21     0.077599 -0.237126       2        1  12.112956
3      0.129467  0.157677       3        1  10.146523
24     0.203014 -0.134047       4        1   8.249854
6      0.013290 -0.191501       5        1   7.480332
18     0.167210 -0.066079       6        1   6.985654
23    -0.002056 -0.119837       7        1   3.074063
12     0.073095  0.121375       8        1   2.904516
0     -0.011326  0.121997       9        1   2.798772
4      0.086433 -0.048434      10        1   2.519907
10     0.142515 -0.042090      11        1   2.035255
19     0.076838  0.042932      12        1   1.958947
8      0.076067  0.239129      13        1   1.767550
1      0.108399  0.070844      14        1   1.577579
13     0.065893 -0.039977      15        1   1.408343
17    -0.109301 -0.066963      16        1   1.272033
22     0.058481  0.059318      17        1   1.189299
14    -0.169860 -0.040851      18        1   0.672922
20    -0.172047  0.043095      19        1   0.604208
2     -0.076379  0.073823      20        1   0.570574
16    -0.137435  0.001546      21        1   0.555212
11    -0.178169 -0.012579      22        1   0.508332
5     -0.208017  0.001329      23        1   0.465735
9     -0.211896  0.010764      24        1   0.254437
7     -0.208395  0.002252      25        1   0.077553, topic_info=         Term         Freq        Total Category  logprob  loglift
60        n't  1877.000000  1877.000000  Default  30.0000  30.0000
130     would  1281.000000  1281.000000  Default  29.0000  29.0000
123     think   809.000000   809.000000  Default  28.0000  28.0000
214     space   835.000000   835.000000  Default  27.0000  27.0000
723       god   707.000000   707.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
5210     comp     0.013015     0.908576  Topic25  -9.2688   2.9162
1860   machin     0.013015    84.577217  Topic25  -9.2688  -1.6173
1855    liber     0.013015    10.591258  Topic25  -9.2688   0.4603
1854  lengthi     0.013015     4.883894  Topic25  -9.2688   1.2344
1852   legisl     0.013015    20.195255  Topic25  -9.2688  -0.1851

[1412 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4245     11  0.943075        a.a
3445     22  0.661066   a.lizard
4377      8  0.970402      abort
399       1  0.977742    absolut
179      22  0.962849    acronym
...     ...       ...        ...
617      14  0.032571       year
3725      7  0.939855        yee
4204     13  0.924358  yesterday
3606     10  0.976748       zero
4345     14  0.848744       zone

[2731 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 22, 4, 25, 7, 19, 24, 13, 1, 5, 11, 20, 9, 2, 14, 18, 23, 15, 21, 3, 17, 12, 6, 10, 8])